# Day 17: Trick Shot

In [1]:
from pathlib import Path
import re
from dataclasses import dataclass
from typing import Iterable
from functools import lru_cache
from itertools import count
from more_itertools import pairwise, first, ilen

from aoc2021.util import read_as_str, natsum

## Puzzle input data

In [2]:
parse_input = lambda x: tuple(map(int, re.findall('[\-0-9]+', x.rstrip())))
# Test data.
tdata = parse_input('target area: x=20..30, y=-10..-5')

# Input data.
data = parse_input(read_as_str(Path('./day17-input.txt')))
data

(60, 94, -171, -136)

## Puzzle answers
### Part 1

In [3]:
@dataclass(frozen=True, eq=True)
class Pos:
    x: int  # horizontal (+ve forward)
    y: int  # vertical (+ve upward)

    def __add__(self, p):
        return Pos(self.x + p.x, self.y + p.y)


@dataclass(frozen=True, eq=True)
class Vel(Pos):

    def __add__(self, v):
        return Vel(self.x + v.x, self.y + v.y)


@dataclass(frozen=True, eq=True)
class Trajectory:
    v0: Vel
    p0: Pos = Pos(0,0)
    
    @property
    @lru_cache
    def ymax(self):
        return first(self.pos(s1).y for s1,s2 in pairwise(count()) if self.pos(s1).y >= self.pos(s2).y)

    @lru_cache
    def vel(self, step: int) -> Vel:
        if step == 0:
            return self.v0
        v = self.vel(step-1)
        return v + Vel(-v.x//abs(v.x) if v.x != 0 else 0, -1)

    @lru_cache
    def pos(self, step: int) -> Pos:
        if step == 0:
            return self.p0
        return self.pos(step-1) + self.vel(step-1)


@dataclass(frozen=True, eq=True)
class Target:
    xmin: int
    xmax: int
    ymin: int
    ymax: int


def in_target(pos: Pos, tgt: Target) -> bool:
    return tgt.xmin <= pos.x <= tgt.xmax and tgt.ymin <= pos.y <= tgt.ymax


def past_target(pos: Pos, tgt: Target) -> bool:
    return pos.x > tgt.xmax or pos.y < tgt.ymin


def to_target(tr: Trajectory, tgt: Target) -> bool:
    ps = (tr.pos(step) for step in count())
    while True:
        p = next(ps)
        if in_target(p, tgt):
            return True
        if past_target(p, tgt):
            return False


def target_trajectories(tgt: Target, p0: Pos = Pos(0,0)) -> Iterable[Trajectory]:
    for vx in count(start=first(n for n in count() if natsum(n) >= tgt.xmin)):
        for vy in count(start=tgt.ymin):
            tr = Trajectory(Vel(vx, vy), p0=p0)
            if to_target(tr, tgt):
                yield tr
            if vy > 300:
                break
        if vx > tgt.xmax:
            break


def max_height(tgt: Target) -> Vel:
    return max(tr.ymax for tr in target_trajectories(tgt))


assert Pos(1,1) + Pos(-1,2) == Pos(0,3)
assert Pos(1,1) + Vel(-1,2) == Pos(0,3)
assert [Trajectory(Vel(2,3)).pos(s) for s in range(5)] == [Pos(0,0), Pos(2,3), Pos(3,5), Pos(3,6), Pos(3,6)]
assert [Trajectory(Vel(2,3)).vel(s) for s in range(5)] == [Vel(2,3), Vel(1,2), Vel(0,1), Vel(0,0), Vel(0,-1)]
assert [Trajectory(Vel(7,2)).pos(s) for s in range(5)] == [Pos(0,0), Pos(7,2), Pos(13,3), Pos(18,3), Pos(22,2)]
assert [Trajectory(Vel(7,2)).vel(s) for s in range(5)] == [Vel(7,2), Vel(6,1), Vel(5,0), Vel(4,-1), Vel(3,-2)]
assert Trajectory(Vel(2,3)).ymax == 6
assert in_target(Pos(20,-7), Target(*[20,30,-10,-5])) == True
assert in_target(Pos(20,-11), Target(*[20,30,-10,-5])) == False
assert all(to_target(Trajectory(Vel(*v)), Target(*tdata)) for v in [(7,2),(6,3),(9,0)]) == True
assert to_target(Trajectory(Vel(17,-4)), Target(*tdata)) == False
assert max_height(Target(*tdata)) == 45

In [4]:
n = max_height(Target(*data))
print(f'The highest y position the probe reaches on its trajectory: {n}')

The highest y position the probe reaches on its trajectory: 14535


### Part 2

In [5]:
assert ilen(target_trajectories(Target(*tdata))) == 112

In [6]:
n = ilen(target_trajectories(Target(*data)))
print(f'Number of distinct initial velocity values that lead the probe to the target area: {n}')

Number of distinct initial velocity values that lead the probe to the target area: 2270
